In [ ]:
# imports
import AmpliPy
import pysam
import multiprocessing

MAIN_EXAMPLE_INPUT_FILE = "./example/example_untrimmed_sorted.bam"

In [ ]:
# simple function to take time
import time
def func_to_run(read):
	time.sleep(.5)

In [ ]:
# counts total reads we have
import AmpliPy
in_align,_ = AmpliPy.create_AlignmentFile_objects(MAIN_EXAMPLE_INPUT_FILE)
count = 0
for read in in_align:
	count += 1

print("total reads", str(count)) # last count was 201600

In [ ]:
# creates a smaller version of the file
import os
new_input_file = "intermediary.bam"
os.remove(new_input_file) # you need a new file each time

in_align,out_align = AmpliPy.create_AlignmentFile_objects(MAIN_EXAMPLE_INPUT_FILE, new_input_file)

i = 0
for read in in_align:
	if (i >= 100):
		break
	out_align.write(read)
	i += 1
out_align.close()

In [ ]:
# read in the newly created file, ensuring we created the right file size
inter_align,_ = AmpliPy.create_AlignmentFile_objects(new_input_file)

count = 0
for read in inter_align:
	count += 1

print("count: ", str(count)) # count was 100 on last run


In [ ]:
# simple test to check time for a linear approach
inter_align,_ = AmpliPy.create_AlignmentFile_objects(new_input_file)

linear_start = time.time()
for read in inter_align:
	func_to_run(read)
linear_end = time.time()

linear_time = linear_end - linear_start
print("total time for linear approach: ", str(linear_time)) # took 50.36s on last run

In [48]:
# simple multiprocessing test
import multiprocessing
inter_align,_ = AmpliPy.create_AlignmentFile_objects(new_input_file)

for read in inter_align:
	fake_read = {
		'cigartuples': read.cigartuples,
		'is_paired': read.is_paired,
		'is_reverse': read.is_reverse,
		'query_alignment_qualities': read.query_alignment_qualities,
		'query_alignment_start': read.query_alignment_start,
		'query_length': read.query_length,
		'reference_end': read.reference_end,
		'reference_start': read.reference_start,
		'template_length': read.template_length
	}